In [59]:
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import PIL
from torchvision.transforms.autoaugment import CIFAR10

ImportError: cannot import name 'CIFAR10' from 'torchvision.transforms.autoaugment' (/opt/anaconda3/lib/python3.11/site-packages/torchvision/transforms/autoaugment.py)

In [33]:
PATH = 'dataset'

In [34]:
def verify_image(fp):
    try:
        PIL.Image.open(fp).verify()
        return True
    except:
        return False

In [40]:
transform = transforms.Compose([
    transforms.Resize(size=100),
    transforms.RandomRotation(degrees=180),
    transforms.AutoAugment(),
    transforms.ToTensor()
    ImageNetPolicy(),
])

In [52]:
dataset = datasets.ImageFolder(root=PATH, is_valid_file=verify_image)
print(len(dataset))

612


In [45]:
aa_transform = transforms.AutoAugment()
aa_transform.subpolicies = transform
dataset = transforms.Compose([
                                        aa_transform,
                                        transforms.ToTensor()
                                    ])

In [47]:
print(dataset)

Compose(
    AutoAugment(policy=AutoAugmentPolicy.IMAGENET, fill=None)
    ToTensor()
)
